## Filtering Data

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [4]:
# Order a DataFrame by ascending values
df.orderBy(df["points"].asc()).show(5)

+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|country|         description|         designation|points|price|province|          region_1|      region_2|variety|      winery|
+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|     US|The strong aroma ...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|This offers gener...|                null|  null| null|    null|              null|          null|   null|        null|
|     US|This shows a deft...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|"This nicely stru...| bitter almond an...|  null|   88|      24|Northeastern Italy|Venezia Giulia|   null|Pinot Grigio|
|     US|               Ripe |                null|  null| null|    null|              null|     

In [5]:
# Order a DataFrame by descending values
df.orderBy(df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|   region_1|region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|     US|This is an absolu...|           IX Estate|    99|  290|California|Napa Valley|    Napa|           Red Blend|              Colgin|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|  Bordeaux|   Pauillac|    null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
|     US|There are incredi...|Elevation 1147 Es...|    99|  150|California|Napa Valley|    Napa|  Cabernet Sauvignon|        David Arthur|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385| Champagne|  Champagne|    null|     Champagne Blend|     Mo��t & Chandon|
|  Italy|Even better than .

In [6]:
# Import average function
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [7]:
# Using filter
df.filter("price<20").show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
| Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|    Spain|Earthy plum and c...|              Amandi|    90|   17|         Galicia|       Ribeira Sacra|                null|             Menc�_a|      Don Bernardino|
|       US|There's a lot to ...|                null|    90|   18|      California|Russian River Valley|              Sonoma|          Chardonnay|            De

In [8]:
# Filter by price on certain columns
df.filter("price<20").select(['points','country', 'winery','price']).show()

+------+---------+--------------------+-----+
|points|  country|              winery|price|
+------+---------+--------------------+-----+
|    90| Bulgaria|        Villa Melnik|   15|
|    90|    Spain|      Don Bernardino|   17|
|    90|       US|            De Loach|   18|
|    91|       US|   Trinity Vineyards|   19|
|    91| Portugal|Adega Cooperativa...|   15|
|    86|       US|      Belle Ambiance|   10|
|    86| Portugal| Adega de Cantanhede|   12|
|    86|       US|            Parducci|   13|
|    86| Portugal|    Quinta do Portal|   10|
|    86|   France|               Rigal|   14|
|    86|       US|     The Naked Grape|   18|
|    86|   France|   Georges Vigouroux|   15|
|    86|   France|   Georges Vigouroux|   10|
|    86|       US| Martinez & Martinez|   17|
|    86|       US|           Ironstone|   12|
|    86|       US|       Leaping Horse|   10|
|    86|       US|        Kitchen Sink|   13|
|    86| Portugal|  Wines & Winemakers|   12|
|    86|Argentina|              Zo

### Using Python Comparison Operators

In [9]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|          region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|              Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|    Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|            Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
| France|This is the top w...|    

In [10]:
df.filter( (df["price"] < 200) | (df['points'] > 80) ).show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [11]:
df.filter(df["country"] == "US").show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Or